In [1]:
import pandas as pd
import numpy as np

import time

from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 42

In [3]:
raw_df = pd.read_csv('./data/boston.csv')
raw_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MDEV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
# 
X = raw_df.iloc[:,0:14]
y = raw_df.iloc[:,13:14]

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
regressions = [LinearRegression(),
               Lasso(),
               Ridge()
               ]

for regression in regressions:
    start_time = time.time()
    pipe = Pipeline(steps=[('regression', regression)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    elapsed_time = time.time() - start_time
    print('Модель: ',regression)
    print('Время выполнения: %.3f' %elapsed_time)
    print('Коэффициент регрессии',round(r2_score(y_test, y_pred),10))
    print('----')

Модель:  LinearRegression()
Время выполнения: 0.014
Коэффициент регрессии 1.0
----
Модель:  Lasso()
Время выполнения: 0.004
Коэффициент регрессии 0.9998428245
----
Модель:  Ridge()
Время выполнения: 0.004
Коэффициент регрессии 0.9999999958
----


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
coef_reg = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]


In [8]:
regressions = [ Lasso,
                Ridge
               ]

In [9]:
for regression in regressions:

    params = [{'alpha': coef_reg}]
    search = GridSearchCV(regression(), 
                          params,
                          scoring='r2',
                          cv=5).fit(X_train, y_train)

    model = regression(search.best_params_['alpha']).fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print('Модель: ',regression())
    print('Коэффициент регуляризации: ', search.best_params_['alpha'])
    print('Коэффициент регрессии: ',round(r2_score(y_test, y_pred),10))
    print('----')
    



Модель:  Lasso()
Коэффициент регуляризации:  1e-05
Коэффициент регрессии:  0.9999999986
----
Модель:  Ridge()
Коэффициент регуляризации:  1e-05
Коэффициент регрессии:  1.0
----


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [11]:
regressions = [ Lasso,
                Ridge
               ]

In [12]:
transformer = Pipeline(
                       [
                           ('MinMax', MinMaxScaler()),
                           ('StandartScaler', StandardScaler())
                          
                       ] 
                      )

In [13]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

In [14]:
preprocessor = ColumnTransformer(transformers=[
                                 'scaler', transformer, numeric_features
                                ])                           

In [15]:
pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('regression', Lasso())
    ]
)

In [20]:
model = pipe.fit(X_train, y_train)
y_pred = model.predict(y_test)

ValueError: not enough values to unpack (expected 3, got 2)

In [19]:
coef_reg = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]

params = [{'regression__alpha': coef_reg}]

for regression in regressions:
    gs_= GridSearchCV(
                        pipe,
                        param_grid=params,
                        scoring='r2',
                        cv=5,
                        n_jobs=-1,
                        verbose=2
)

    model = gs_.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Модель: ',regression)
    print('Время выполнения: %.3f' %elapsed_time)
    print('Коэффициент регрессии',round(r2_score(y_test, y_pred),10))
    print('----')

Fitting 5 folds for each of 11 candidates, totalling 55 fits


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
model = gs_lasso.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print('Коэффициент регрессии: ',round(r2_score(y_test, y_pred),10))

In [ ]:
scaler = MinMaxScaler()
pipe = scaler.fit(X_train, y_train)
# pipe.transform(X_train, y_train)

In [ ]:
pipe.get_params()

In [ ]:
numeric_features

In [ ]:
model = pipe.fit(X_train, y_train)

In [ ]:
regressions = [ Lasso,
                Ridge
               ]

for regression in regressions:
    start_time = time.time()
    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed_time = time.time() - start_time
    print('Модель: ',regression)
    print('Время выполнения: %.3f' %elapsed_time)
    print('Коэффициент регрессии',round(r2_score(y_test, y_pred),10))
    print('----')

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [21]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [22]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [23]:
map_target = {'<=50K' : '0',
              '>50K' : '2'}

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [24]:
X = data.iloc[:, 0:14]
y = data.iloc[:, 14:15]

In [25]:
# y = y.map(map_target)
y['Target'] = y[14].map(map_target)

In [26]:
y.drop(columns=[14], inplace=True)

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       48842 non-null  int64 
 1   1       48842 non-null  object
 2   2       48842 non-null  int64 
 3   3       48842 non-null  object
 4   4       48842 non-null  int64 
 5   5       48842 non-null  object
 6   6       48842 non-null  object
 7   7       48842 non-null  object
 8   8       48842 non-null  object
 9   9       48842 non-null  object
 10  10      48842 non-null  int64 
 11  11      48842 non-null  int64 
 12  12      48842 non-null  int64 
 13  13      48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


10. Выберите колонки с числовыми и категориальными переменными.

In [73]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['category']).columns

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [34]:
from sklearn.preprocessing import OneHotEncoder

In [35]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [36]:
from sklearn.linear_model import LogisticRegression

In [74]:
num_pipe = Pipeline(
                    ('MinMaxScaler', MinMaxScaler())
                    )

cat_pipe = Pipeline(
                    ('OneHotEncoder', OneHotEncoder())
                    )

num = ColumnTransformer(transformers=['Scaler', num_pipe, numeric_features])
cat = ColumnTransformer(transformers=['Encode', cat_pipe, categorical_features])

full_pipe = Pipeline(['feat_union', FeatureUnion(transformer_list=[('numeric', num),('categorical', cat)]),'LRegr', LogisticRegression() ])

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [79]:
num_pipe.fit(X_train[numeric_features], y_train)

ValueError: too many values to unpack (expected 2)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.